In [1]:
!pip3 install -q -U google-generativeai

In [21]:
import time
from tqdm import tqdm
import json
from dotenv import load_dotenv
import os
from tqdm import tqdm
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY2")

In [22]:
import google.generativeai as genai
from IPython.display import Image, display
import PIL.Image
import pandas as pd

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [23]:
# Set model parameters
generation_config = genai.GenerationConfig(
    temperature=0.8,
)

# Set safety settings
safety_settings = [
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
]

In [26]:
df = pd.read_csv("csv_files/annotated_corrected_with_contexts.csv")

In [27]:
df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?
0,1dncran,https://i.redd.it/1xvccvq6oi8d1.jpeg,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,1
1,c9tdn7,https://i.imgur.com/6KzNep5.jpg,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,1
2,kctb6e,https://i.redd.it/9knprglwx3561.jpg,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,1,1
3,oagyyo,https://i.redd.it/dm6sx4nqn9871.jpg,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1
4,faw3bs,https://i.imgur.com/gwVDXvB.jpg,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1


### Initializing new columns for llm generated annotations

In [29]:
df['is_1'] = None
df['is_2'] = None
df['is_3'] = None
df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3
0,1dncran,https://i.redd.it/1xvccvq6oi8d1.jpeg,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,1,None,None,None
1,c9tdn7,https://i.imgur.com/6KzNep5.jpg,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,1,None,None,None
2,kctb6e,https://i.redd.it/9knprglwx3561.jpg,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,1,1,None,None,None
3,oagyyo,https://i.redd.it/dm6sx4nqn9871.jpg,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1,None,None,None
4,faw3bs,https://i.imgur.com/gwVDXvB.jpg,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1,None,None,None


In [30]:
sampled_df = df.sample(n=100, random_state=42).reset_index(drop=True)
sampled_df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3
0,csgp00,https://i.redd.it/z8ey88ejkeh31.jpg,dog injected full of drugs and kidnapped,it's full of doguments.,NaN,the look on the dogs face shows that even they...,1,0,1,None,None,None
1,bwmohn,https://i.redd.it/6roixwhh5b231.jpg,sick man cuts poor long dog in half and it fuc...,poor doggo lost his bark,post this on r/confusing_perspective,do ggo,1,0,1,None,None,None
2,drrqty,https://i.imgur.com/7mkUGwN.jpg,for real tho,the world's oldest teenager,bet he got a splinter here and there,fun fact: michelangelo used a unique tool he a...,0,1,1,None,None,None
3,uzn0h9,https://i.redd.it/s6fwn5ngu7291.jpg,welcome to america,link? i searched that website for articles wit...,"nimby, nothing new",what do people think the alternative for homel...,1,0,1,None,None,None
4,h9nea0,https://i.redd.it/w86h3pibo4551.jpg,you could say that the jedi's mind was truly.....,NaN,"high midichlorine count != jedi/sith, just the...","there was a comic in ledgends about this, i do...",0,1,1,None,None,None


In [31]:
sampled_df.shape

(100, 12)

In [92]:
def provide_prompt_for_image_labels():
    image_prompt = f"""
    TASK:
    You will be given an image, and your task is to determine whether the image contains sarcasm. 
    Indicate your response with a 1 if sarcasm is detected and a 0 if it is not sarcastic.

    To give you an idea:
    Sarcasm is a subtle form of language in which people express the opposite of what is implied.
    Th image (with or without the help of text on it) must imply a strong sarcastic irony, insult, or juxtaposition.
    If the image fails to do so , it is non sarcastic by default.
    Most images are non sarcastic.

    EXAMPLES:
    """
    examples = [
        ("Sample Images/oil_painting.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/cable.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/tumor.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/snake.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/ayezy8.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/Traffic_bounded.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/epymwl.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/university.png", '{"isImageSarcastic?": 0}'),
    ]

    message = [image_prompt]
    for image, resp in examples:
        message = message + [
            PIL.Image.open(image),
            resp
        ]

    return message

# Defining a response schema in order to obtain consistently formatted response
response_schema_image = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "isImageSarcastic?": {
                "type": "integer",
            },
        },
        "required": ["isImageSarcastic?"],
    },
}

def detect_sarcasm_in_image(image:str) -> str:
    response = model.generate_content(
        provide_prompt_for_image_labels() + [
            PIL.Image.open(image)
        ],
        safety_settings=safety_settings,
        generation_config = genai.GenerationConfig(
            response_mime_type="application/json", response_schema=response_schema_image, temperature = 0.8
        )
    )
    return response.text

In [93]:
IMAGE_DIRECTORY = "/Users/sinngamkhaidem/Developer/Datasets/osint mmsd/annotated/images"

In [94]:
curIndex = 0

In [95]:
curIndex

0

In [96]:
for i in tqdm(range(curIndex, sampled_df.shape[0])):
    try:
        img = os.path.join(IMAGE_DIRECTORY, f"{sampled_df.iloc[i]['submissionId']}.png")
        
        # Attempt to detect sarcasm in the image
        response = detect_sarcasm_in_image(img)
        
        # Parse the response and update the DataFrame
        response_as_dict = json.loads(response)
        if len(response_as_dict) > 0:
            sampled_df.at[i, 'is_3'] = response_as_dict[0].get('isImageSarcastic?',0)
            
    except Exception as e:
        sampled_df.at[i, 'is_3'] = 0
        
    finally:
        # Always update curIndex to ensure progress is saved
        curIndex = i

100%|██████████| 100/100 [07:33<00:00,  4.54s/it]


In [97]:
sampled_df['isImageSarcastic?'].value_counts()

isImageSarcastic?
0    50
1    50
Name: count, dtype: int64

In [98]:
sampled_df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3
0,csgp00,https://i.redd.it/z8ey88ejkeh31.jpg,dog injected full of drugs and kidnapped,it's full of doguments.,NaN,the look on the dogs face shows that even they...,1,0,1,1,1,1
1,bwmohn,https://i.redd.it/6roixwhh5b231.jpg,sick man cuts poor long dog in half and it fuc...,poor doggo lost his bark,post this on r/confusing_perspective,do ggo,1,0,1,0,0,0
2,drrqty,https://i.imgur.com/7mkUGwN.jpg,for real tho,the world's oldest teenager,bet he got a splinter here and there,fun fact: michelangelo used a unique tool he a...,0,1,1,1,1,1
3,uzn0h9,https://i.redd.it/s6fwn5ngu7291.jpg,welcome to america,link? i searched that website for articles wit...,"nimby, nothing new",what do people think the alternative for homel...,1,0,1,1,1,1
4,h9nea0,https://i.redd.it/w86h3pibo4551.jpg,you could say that the jedi's mind was truly.....,NaN,"high midichlorine count != jedi/sith, just the...","there was a comic in ledgends about this, i do...",0,1,1,1,1,1


In [99]:
sampled_df.is_3.value_counts()

is_3
1    68
0    32
Name: count, dtype: int64

# Evaluation

In [100]:
from sklearn.metrics import cohen_kappa_score
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    kappa = cohen_kappa_score(y_true, y_pred)

    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1,"\n", "\tCohen's Kappa Score: ", kappa)

In [101]:
evaluate(sampled_df['isImageSarcastic?'].to_list(), sampled_df['is_3'].to_list())

	Accuracy:  0.8 
 	Precision:  0.7205882352941176 
 	Recall:  0.98 
 	F1 score:  0.8305084745762712 
 	Cohen's Kappa Score:  0.6


In [102]:
sampled_df.to_csv('labelled_csv/gemini-image-labels.csv')